In [ ]:
# %%
# Imports (same as before, plus nltk/textblob for sentiment)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import shap
import warnings
import datetime
import sys
warnings.filterwarnings('ignore')

try:
    import yfinance as yf
    YF_AVAILABLE = True
except ImportError:
    YF_AVAILABLE = False

try:
    from transformers import pipeline
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    TRANSFORMERS_AVAILABLE = False

try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    VADER_AVAILABLE = True
except ImportError:
    VADER_AVAILABLE = False

try:
    from textblob import TextBlob
    TEXTBLOB_AVAILABLE = True
except ImportError:
    TEXTBLOB_AVAILABLE = False


In [ ]:
# %%
# Constants and global parameters
np.random.seed(42)
N_DAYS = 500
SYMBOLS = ['XOP', 'XLE', 'USO', 'BNO', 'SPY', 'VIX']
N_FEATURES = len(SYMBOLS)
N_COMPONENTS = 5
ROLLING_WINDOW = 30
START_DATE = '2023-01-01'
END_DATE = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
# %%
# Data loading: Try real data, else fallback to synthetic
def fetch_real_data(symbols, start_date, end_date):
    if not YF_AVAILABLE:
        return None
    try:
        data = yf.download(symbols, start=start_date, end=end_date)['Close']
        if isinstance(data, pd.Series):
            data = data.to_frame()
        data = data.dropna(axis=1, how='all').dropna(axis=0, how='all')
        if data.empty or data.shape[0] < 30:
            return None
        return data
    except Exception as e:
        print(f"[WARN] Could not fetch real data: {e}")
        return None

In [ ]:
def generate_synthetic_data(symbols, n_days):
    dates = pd.date_range(start=START_DATE, periods=n_days, freq='B')
    data = np.cumsum(np.random.randn(n_days, len(symbols)) * 0.01 + 0.0005, axis=0) + 100
    return pd.DataFrame(data, index=dates, columns=symbols)

prices_df = fetch_real_data(SYMBOLS, START_DATE, END_DATE)
if prices_df is None:
    print("[INFO] Using synthetic data.")
    prices_df = generate_synthetic_data(SYMBOLS, N_DAYS)
else:
    print("[INFO] Using real market data.")

returns_df = prices_df.pct_change().dropna()

# %%
# Data preprocessing: handle missing values, standardize
def preprocess_returns(df):
    df = df.dropna(axis=1, how='all').dropna(axis=0, how='all')
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df)
    return pd.DataFrame(scaled, index=df.index, columns=df.columns), scaler

returns_scaled, scaler = preprocess_returns(returns_df)

In [ ]:
# %%
# Advanced Sentiment Analyzer
class AdvancedSentimentAnalyzer:
    def __init__(self):
        if VADER_AVAILABLE:
            self.analyzer = SentimentIntensityAnalyzer()
        else:
            self.analyzer = None
    def analyze(self, text):
        if self.analyzer:
            return self.analyzer.polarity_scores(text)['compound']
        elif TEXTBLOB_AVAILABLE:
            return TextBlob(text).sentiment.polarity
        elif TRANSFORMERS_AVAILABLE:
            pipe = pipeline('sentiment-analysis')
            return pipe(text)[0]['score'] * (1 if pipe(text)[0]['label'] == 'POSITIVE' else -1)
        else:
            # Fallback: random sentiment
            return np.random.uniform(-1, 1)

sentiment_analyzer = AdvancedSentimentAnalyzer()

# %%
# Generate synthetic news headlines for each day (for demo)
news_df = pd.DataFrame({
    'date': returns_df.index,
    'headline': [f"Oil market update {i}" for i in range(len(returns_df))],
    'source': np.random.choice(['Reuters', 'Bloomberg', 'BBC', 'Unknown'], size=len(returns_df), p=[0.3,0.3,0.2,0.2])
})
news_df['sentiment'] = news_df['headline'].apply(sentiment_analyzer.analyze)

# --- News Source Reliability Weighting ---
source_reliability = {'Reuters': 1.0, 'Bloomberg': 0.95, 'BBC': 0.9, 'Unknown': 0.5}
news_df['reliability'] = news_df['source'].map(source_reliability).fillna(0.5)
news_df['weighted_sentiment'] = news_df['sentiment'] * news_df['reliability']

# --- Agentic AI System: Memory, Novelty Detection, Rationale Logging ---
from collections import deque

def jaccard_similarity(a, b):
    set_a, set_b = set(a.lower().split()), set(b.lower().split())
    return len(set_a & set_b) / (len(set_a | set_b) + 1e-6)

agent_memory = deque(maxlen=1000)  # store past headlines
novelty_flags = []
llm_rationales = []
for idx, row in news_df.iterrows():
    # Novelty detection: compare to memory
    similarities = [jaccard_similarity(row['headline'], h) for h in agent_memory]
    is_novel = (max(similarities, default=0) < 0.5)
    novelty_flags.append(is_novel)
    agent_memory.append(row['headline'])
    # LLM rationale (placeholder)
    rationale = f"{'Novel' if is_novel else 'Known'} event from {row['source']} (reliability={row['reliability']:.2f}): sentiment={row['sentiment']:.2f}"
    llm_rationales.append(rationale)
news_df['is_novel'] = novelty_flags
news_df['llm_rationale'] = llm_rationales

# --- Audit Trail ---
audit_trail = news_df[['date', 'headline', 'source', 'sentiment', 'reliability', 'is_novel', 'llm_rationale']].copy()
audit_trail.to_csv('news_audit_trail.csv', index=False)

# --- Aggregate daily sentiment (reliability-weighted, novelty-aware) ---
def aggregate_sentiment(df):
    # Optionally upweight novel, reliable headlines
    weights = df['reliability'] * (1.2 if df['is_novel'].any() else 1.0)
    return np.average(df['sentiment'], weights=weights)

daily_sentiment = news_df.groupby('date').apply(aggregate_sentiment)
daily_reliability = news_df.groupby('date')['reliability'].mean()

# --- Sentiment Volatility Features ---
sentiment_features = pd.DataFrame({'agg_sentiment': daily_sentiment, 'avg_reliability': daily_reliability})
sentiment_features['rolling_std'] = sentiment_features['agg_sentiment'].rolling(window=3).std()
sentiment_features['rolling_range'] = sentiment_features['agg_sentiment'].rolling(window=3).apply(lambda x: x.max() - x.min())
sentiment_features['rolling_trend'] = sentiment_features['agg_sentiment'].diff(periods=3)
# Down-weight or veto signals on high volatility days
vol_threshold = sentiment_features['rolling_std'].quantile(0.75)
sentiment_features['signal_weight'] = np.where(sentiment_features['rolling_std'] > vol_threshold, 0, 1)


In [ ]:
# --- Enhanced Multi-criteria Signal Generator ---
def generate_signals(returns_df, sentiment_features):
    signals = pd.DataFrame(index=returns_df.index)
    # Momentum: 5-day return
    signals['momentum'] = returns_df['XOP'].rolling(5).mean()
    # Volatility: 5-day std
    signals['volatility'] = returns_df['XOP'].rolling(5).std()
    # Sentiment: align with new features
    signals['sentiment'] = sentiment_features['agg_sentiment'].reindex(returns_df.index)
    signals['sentiment_vol'] = sentiment_features['rolling_std'].reindex(returns_df.index)
    signals['sentiment_trend'] = sentiment_features['rolling_trend'].reindex(returns_df.index)
    signals['avg_reliability'] = sentiment_features['avg_reliability'].reindex(returns_df.index)
    signals['signal_weight'] = sentiment_features['signal_weight'].reindex(returns_df.index)
    # Composite signal: only if sentiment volatility is low, reliability is high
    reliability_cutoff = 0.7
    sentiment_threshold = 0.1
    signals['composite'] = (
        0.3 * signals['momentum'].fillna(0) +
        0.3 * signals['sentiment'].fillna(0) * (signals['avg_reliability'] > reliability_cutoff) * (signals['signal_weight']) +
        0.2 * signals['avg_reliability'].fillna(0) +
        0.2 * (signals['sentiment_vol'] < vol_threshold).astype(float)
    )
    # Final trading signal: buy if composite > 0, sell if < 0, veto if signal_weight==0
    signals['signal'] = np.where((signals['composite'] > 0) & (signals['signal_weight'] > 0), 1, -1)
    return signals

signals = generate_signals(returns_df, sentiment_features)

# --- Model Drift Monitoring ---
# Rolling hit rate (signal accuracy)
signals['hit'] = (signals['signal'] * returns_df['XOP'] > 0).astype(int)
signals['rolling_hit_rate'] = signals['hit'].rolling(window=60).mean()
plt.figure(figsize=(10,3))
plt.plot(signals['rolling_hit_rate'], label='Rolling Hit Rate (Model Drift)')
plt.axhline(0.5, color='red', linestyle='--', label='Random Baseline')
plt.title('Model Drift Monitoring: Rolling Hit Rate')
plt.legend()
plt.show()

# --- Governance & Ethics (comments only) ---
# - Only headlines and public summaries are used (no paywalled content)
# - Reliability scores are based on public bias charts and backtests
# - Multi-source data is used to mitigate bias
# - Model drift is monitored and triggers recalibration if hit rate drops


In [ ]:
# %%
# Risk Manager (position sizing, stop-loss, max drawdown control)
def apply_risk_management(signals, returns_df, max_dd_limit=0.2, stop_loss=0.05):
    equity = [1]
    drawdown = [0]
    for i in range(1, len(signals)):
        ret = signals['signal'].iloc[i-1] * returns_df['XOP'].iloc[i]
        new_equity = equity[-1] * (1 + ret)
        # Stop-loss
        if ret < -stop_loss:
            new_equity = equity[-1] * (1 - stop_loss)
        # Max drawdown control
        max_equity = max(equity)
        dd = (new_equity / max_equity) - 1
        if dd < -max_dd_limit:
            new_equity = max_equity * (1 - max_dd_limit)
        equity.append(new_equity)
        drawdown.append(dd)
    signals['equity_curve'] = equity
    signals['drawdown'] = drawdown
    return signals

signals = apply_risk_management(signals, returns_df)


In [ ]:
# %%
# Bias Detection (mean/median bias, regime bias, feature bias)
def detect_bias(signals, returns_df):
    bias_report = {}
    # Mean/median bias
    bias_report['mean_signal'] = signals['signal'].mean()
    bias_report['median_signal'] = signals['signal'].median()
    # Regime bias: compare signal mean in up vs down markets
    up_market = returns_df['XOP'] > 0
    bias_report['signal_in_up_market'] = signals['signal'][up_market].mean()
    bias_report['signal_in_down_market'] = signals['signal'][~up_market].mean()
    # Feature bias: correlation with sentiment, momentum, volatility
    bias_report['corr_sentiment'] = signals['signal'].corr(signals['sentiment'])
    bias_report['corr_momentum'] = signals['signal'].corr(signals['momentum'])
    bias_report['corr_volatility'] = signals['signal'].corr(signals['volatility'])
    return bias_report

bias_report = detect_bias(signals, returns_df)
print("\n--- Bias Detection Report ---")
for k, v in bias_report.items():
    print(f"{k}: {v:.4f}")


In [ ]:
# %%
# Optimization (simple grid search for stop-loss and max drawdown)
def optimize_risk_params(signals, returns_df):
    best_return = -np.inf
    best_params = None
    for stop_loss in [0.02, 0.05, 0.1]:
        for max_dd in [0.1, 0.2, 0.3]:
            test_signals = apply_risk_management(signals.copy(), returns_df, max_dd_limit=max_dd, stop_loss=stop_loss)
            final_return = test_signals['equity_curve'][-1] - 1
            if final_return > best_return:
                best_return = final_return
                best_params = {'stop_loss': stop_loss, 'max_dd': max_dd}
    print(f"\n--- Optimization Results ---")
    print(f"Best Return: {best_return:.2%} with params: {best_params}")
    return best_params

best_risk_params = optimize_risk_params(signals, returns_df)


In [ ]:
# %%
# PCA Factor Analysis (same as before)
class FactorAnalyzer:
    def __init__(self, n_components=5):
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
        self.loadings_ = None
        self.explained_variance_ = None
    def fit(self, X):
        self.pca.fit(X)
        self.loadings_ = pd.DataFrame(
            self.pca.components_.T,
            index=X.columns,
            columns=[f'PC{i+1}' for i in range(self.n_components)]
        )
        self.explained_variance_ = pd.Series(
            self.pca.explained_variance_ratio_,
            index=[f'PC{i+1}' for i in range(self.n_components)]
        )
        return self
    def transform(self, X):
        return self.pca.transform(X)
    def summary(self):
        print('Explained variance by component:')
        print(self.explained_variance_)
        print('\nFactor loadings:')
        print(self.loadings_)


In [ ]:

factor_analyzer = FactorAnalyzer(n_components=N_COMPONENTS)
factor_analyzer.fit(returns_scaled)
pca_components = factor_analyzer.transform(returns_scaled)


In [ ]:
# %%
# Visualize explained variance
plt.figure(figsize=(8,4))
plt.bar(range(1, N_COMPONENTS+1), factor_analyzer.explained_variance_)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Explained Variance')
plt.show()

# %%
# Visualize factor loadings
plt.figure(figsize=(10,6))
sns.heatmap(factor_analyzer.loadings_, annot=True, cmap='coolwarm', center=0)
plt.title('PCA Factor Loadings')
plt.show()


In [ ]:
# %%
# Gradient-Boosted Tree + SHAP Analysis
X = returns_scaled.copy()
y = X['XOP'].shift(-1).dropna()
X = X.loc[y.index]
model = GradientBoostingRegressor(n_estimators=100, max_depth=4, random_state=42)
model.fit(X, y)
explainer = shap.Explainer(model)
shap_values = explainer(X)

# %%
# SHAP summary plot
shap.summary_plot(shap_values, X, show=False)
plt.title('SHAP Feature Importance (XOP Next-Day Return)')
plt.show()

In [ ]:
# %%
# Trading Logic: Use optimized risk params
signals = apply_risk_management(signals, returns_df, max_dd_limit=best_risk_params['max_dd'], stop_loss=best_risk_params['stop_loss'])

# Performance metrics
total_return = signals['equity_curve'][-1] - 1
sharpe = np.mean(signals['signal'] * returns_df['XOP']) / np.std(signals['signal'] * returns_df['XOP']) * np.sqrt(252)
max_dd = min(signals['drawdown'])
win_rate = (signals['signal'] * returns_df['XOP'] > 0).mean()

print(f"\n--- Trading Performance (Optimized) ---")
print(f"Total Return: {total_return:.2%}")
print(f"Sharpe Ratio: {sharpe:.2f}")
print(f"Max Drawdown: {max_dd:.2%}")
print(f"Win Rate: {win_rate:.2%}")

# Plot equity curve
plt.figure(figsize=(10,4))
plt.plot(signals['equity_curve'], label='Strategy Equity Curve (Optimized)')
plt.title('Backtest Equity Curve (Optimized)')
plt.xlabel('Date')
plt.ylabel('Equity')
plt.legend()
plt.show()


In [ ]:
# %%
# Advanced Risk Metrics (again, for optimized)
from scipy.stats import skew, kurtosis

volatility = np.std(signals['signal'] * returns_df['XOP']) * np.sqrt(252)
var_95 = np.percentile(signals['signal'] * returns_df['XOP'], 5)
cvar_95 = (signals['signal'] * returns_df['XOP'])[signals['signal'] * returns_df['XOP'] <= var_95].mean()
skewness = skew(signals['signal'] * returns_df['XOP'])
kurt = kurtosis(signals['signal'] * returns_df['XOP'])
turnover = signals['signal'].diff().abs().sum() / len(signals)

print(f"\n--- Advanced Risk Metrics (Optimized) ---")
print(f"Annualized Volatility: {volatility:.2%}")
print(f"VaR (95%): {var_95:.2%}")
print(f"CVaR (95%): {cvar_95:.2%}")
print(f"Skewness: {skewness:.2f}")
print(f"Kurtosis: {kurt:.2f}")
print(f"Turnover: {turnover:.2%}")


In [ ]:
# %%
# Walk-Forward Validation (unchanged)
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)
wf_results = []
for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    model = GradientBoostingRegressor(n_estimators=100, max_depth=4, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    fold_return = np.mean(np.sign(y_pred) * y_test)
    wf_results.append(fold_return)
print(f"\n--- Walk-Forward Validation ---")
print(f"Avg Out-of-Sample Return: {np.mean(wf_results):.4f}")
print(f"Fold Returns: {wf_results}")

# %%
# Enhanced Stress Testing (unchanged)
def enhanced_stress_test(returns, scenarios, window=ROLLING_WINDOW):
    for scenario in scenarios:
        label = scenario['label']
        df = returns.copy()
        if 'shock_col' in scenario and scenario['shock_col'] in df.columns:
            df[scenario['shock_col']] = df[scenario['shock_col']] * (1 + scenario['shock_pct'])
        rolling_drawdown = (df.cumsum() - df.cumsum().cummax())
        rolling_vol = df.rolling(window).std()
        max_drawdown = rolling_drawdown.min().min()
        avg_vol = rolling_vol.mean().mean()
        total_return = df.sum().mean()
        print(f"--- {label} ---")
        print(f"Max Drawdown: {max_drawdown:.2%}")
        print(f"Avg Volatility: {avg_vol:.2%}")
        print(f"Total Return: {total_return:.2%}\n")
        plt.figure(figsize=(10,4))
        plt.plot(rolling_drawdown.index, rolling_drawdown['XOP'], label='XOP Drawdown')
        plt.title(f'{label} - XOP Drawdown')
        plt.xlabel('Date')
        plt.ylabel('Drawdown')
        plt.legend()
        plt.show()

scenarios = [
    {'label': 'COVID Crash', 'shock_col': 'XOP', 'shock_pct': -0.3},
    {'label': 'Oil Spike', 'shock_col': 'USO', 'shock_pct': 0.2},
    {'label': 'SPY Crash', 'shock_col': 'SPY', 'shock_pct': -0.2},
    {'label': 'VIX Spike', 'shock_col': 'VIX', 'shock_pct': 0.5},
]
enhanced_stress_test(returns_df, scenarios)


In [ ]:
# %%
# AI Agent Chain-of-Thought (context-aware, final)
def ai_agent_chain_of_thought(prompt, shap_features, drawdown, sharpe, stress_results, bias_report, best_risk_params):
    summary = f"[AI Agent] Chain-of-thought for: {prompt}\n"
    summary += f"- Top SHAP features: {', '.join(shap_features)}\n"
    summary += f"- Recent max drawdown: {drawdown:.2%}\n"
    summary += f"- Recent Sharpe ratio: {sharpe:.2f}\n"
    summary += f"- Stress test scenarios: {', '.join([s['label'] for s in stress_results])}\n"
    summary += f"- Bias detection: mean={bias_report['mean_signal']:.2f}, median={bias_report['median_signal']:.2f}, up_market={bias_report['signal_in_up_market']:.2f}, down_market={bias_report['signal_in_down_market']:.2f}\n"
    summary += f"- Optimization: best stop_loss={best_risk_params['stop_loss']}, best max_dd={best_risk_params['max_dd']}\n"
    summary += "- Recommendation: Monitor top risk factors, diversify, use robust risk management, and validate with walk-forward and stress tests."
    return summary

shap_importance = np.abs(shap_values.values).mean(axis=0)
top_shap_features = list(X.columns[np.argsort(-shap_importance)][:5])
stress_labels = [s['label'] for s in scenarios]
ai_agent_output = ai_agent_chain_of_thought(
    "Explain the main drivers of XOP's next-day return given the current factor exposures and SHAP analysis.",
    top_shap_features, max_dd, sharpe, scenarios, bias_report, best_risk_params)
print("\n=== AI AGENT CHAIN-OF-THOUGHT (FINAL) ===\n" + ai_agent_output)

# %%
# Final Results Summary
print("\n=== FINAL SUMMARY ===")
factor_analyzer.summary()
print("\nTop SHAP Features for XOP Next-Day Return:")
for i, col in enumerate(top_shap_features):
    print(f"{i+1}. {col} (mean |SHAP|: {shap_importance[np.argsort(-shap_importance)][i]:.4f})")
print("\nTrading performance, risk metrics, walk-forward validation, stress test scenarios, bias detection, optimization, and AI agent reasoning completed. See plots above.") 

In [ ]:
# %%
# AI Agent Chain-of-Thought (context-aware, final)
def ai_agent_chain_of_thought(prompt, shap_features, drawdown, sharpe, stress_results, bias_report, best_risk_params):
    summary = f"[AI Agent] Chain-of-thought for: {prompt}\n"
    summary += f"- Top SHAP features: {', '.join(shap_features)}\n"
    summary += f"- Recent max drawdown: {drawdown:.2%}\n"
    summary += f"- Recent Sharpe ratio: {sharpe:.2f}\n"
    summary += f"- Stress test scenarios: {', '.join([s['label'] for s in stress_results])}\n"
    summary += f"- Bias detection: mean={bias_report['mean_signal']:.2f}, median={bias_report['median_signal']:.2f}, up_market={bias_report['signal_in_up_market']:.2f}, down_market={bias_report['signal_in_down_market']:.2f}\n"
    summary += f"- Optimization: best stop_loss={best_risk_params['stop_loss']}, best max_dd={best_risk_params['max_dd']}\n"
    summary += "- Recommendation: Monitor top risk factors, diversify, use robust risk management, and validate with walk-forward and stress tests."
    return summary

shap_importance = np.abs(shap_values.values).mean(axis=0)
top_shap_features = list(X.columns[np.argsort(-shap_importance)][:5])
stress_labels = [s['label'] for s in scenarios]
ai_agent_output = ai_agent_chain_of_thought(
    "Explain the main drivers of XOP's next-day return given the current factor exposures and SHAP analysis.",
    top_shap_features, max_dd, sharpe, scenarios, bias_report, best_risk_params)
print("\n=== AI AGENT CHAIN-OF-THOUGHT (FINAL) ===\n" + ai_agent_output)

# %%
# Final Results Summary
print("\n=== FINAL SUMMARY ===")
factor_analyzer.summary()
print("\nTop SHAP Features for XOP Next-Day Return:")
for i, col in enumerate(top_shap_features):
    print(f"{i+1}. {col} (mean |SHAP|: {shap_importance[np.argsort(-shap_importance)][i]:.4f})")
print("\nTrading performance, risk metrics, walk-forward validation, stress test scenarios, bias detection, optimization, and AI agent reasoning completed. See plots above.") 